**GANs (Generative Adversarial Networks)**

In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import tensorflow as tf 
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, Dense, LeakyReLU, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam 

In [4]:
mnist = tf.keras.datasets.mnist

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

X_train, X_test = X_train/255.0 * 2 - 1, X_test/255.0 * 2 - 1

In [5]:
N, H, W = X_train.shape
D = H*W
X_train = X_train.reshape(-1, D)
X_test = X_test.reshape(-1, D)

In [6]:
latent_dim = 100

In [7]:
def build_generator(latent_dim):
    i = Input(shape = (latent_dim, ))
    
    x = Dense(256, activation = LeakyReLU(0.2))(i)
    x = BatchNormalization(momentum = 0.8)(x)
    
    x = Dense(512, activation = LeakyReLU(0.2))(x)
    x = BatchNormalization(momentum = 0.8)(x)
    
    x = Dense(1024, activation = LeakyReLU(0.2))(x)
    x = BatchNormalization(momentum = 0.8)(x)
    
    x = Dense(D, activation = 'tanh')(x)
    
    model = Model(i, x)
    return model

In [8]:
def build_discriminator(img_size):
    i = Input(shape = (img_size, ))
    
    x = Dense(512, activation = LeakyReLU(0.2))(i)
    
    x = Dense(256, activation = LeakyReLU(0.2))(x)
    
    x = Dense(1, activation = 'sigmoid')(x)
    
    model = Model(i, x)
    return model

In [9]:
discriminator = build_discriminator(D)
discriminator.compile(loss = 'binary_crossentropy',
                      optimizer = Adam(0.0002, 0.5), 
                      metrics = ['accuracy'])
generator = build_generator(latent_dim)

z = Input(shape = (latent_dim, ))
img = generator(z)

discriminator.trainable = False

fake_pred = discriminator(img)

combined_model = Model(z, fake_pred)
combined_model.compile(loss = 'binary_crossentropy', 
                       optimizer = Adam(0.0002, 0.5), 
                       metrics = ['accuracy'])

2024-08-12 15:23:58.194669: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-12 15:23:58.531518: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-12 15:23:58.531598: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-12 15:23:58.534070: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-12 15:23:58.534199: I tensorflow/compile

In [10]:
import os

bs = 32
epochs = 10000
sample_period = 200

ones = np.ones(bs)
zeros = np.zeros(bs)

d_losses = []
g_losses = []

if not os.path.exists('gan_images'):
    os.makedirs('gan_images')

In [11]:
def sample_images(epoch):
    rows, cols = 5, 5
    noise = np.random.randn(rows * cols, latent_dim)
    imgs = generator.predict(noise)
    
    imgs = 0.5 * imgs + 0.5
    fig, axs = plt.subplots(rows, cols)
    idx = 0
    for i in range(rows):
        for j in range(cols):
            axs[i, j].imshow(imgs[idx].reshape(H, W), cmap='gray')
            axs[i, j].axis('off')
            idx += 1
    fig.savefig('gan_images/%d.png' % epoch)
    plt.close()

In [12]:
for epoch in range(epochs):
    
    #train discriminator
    idx = np.random.randint(0, X_train.shape[0], bs)
    real_imgs = X_train[idx]
    
    noise = np.random.randn(bs, latent_dim)
    fake_imgs = generator.predict(noise)
    
    d_loss_real, d_acc_real = discriminator.train_on_batch(real_imgs, ones) 
    d_loss_fake, d_acc_fake = discriminator.train_on_batch(fake_imgs, zeros)
    d_loss = 0.5 * (d_loss_real + d_loss_fake)
    d_acc = 0.5 * (d_acc_real + d_acc_fake)
    
    
    #train generator
    noise = np.random.randn(bs, latent_dim)
    g_loss = combined_model.train_on_batch(noise, ones)
    
    d_losses.append(d_loss)
    g_losses.append(g_loss)
    
    if epoch%100 == 0:
        print(f'epoch: {epoch + 1}/{epochs}, d_loss: {d_loss}, d_acc: {d_acc}, g_loss: {g_loss}')
        
    if epoch % sample_period == 0:
        sample_images(epoch)

1/1 [==============================] - 1s 511ms/step


2024-08-12 15:23:59.566832: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-08-12 15:24:00.714878: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f800e461590 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-12 15:24:00.714933: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650 Ti, Compute Capability 7.5
2024-08-12 15:24:00.721811: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-12 15:24:00.781404: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-08-12 15:24:00.860562: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


epoch: 1/10000, d_loss: 0.6859928071498871, d_acc: 0.40625, g_loss: [0.6855669021606445, 0.46875]
1/1 [==============================] - 0s 14ms/step
epoch: 101/10000, d_loss: 0.006104327971115708, d_acc: 1.0, g_loss: [4.8524932861328125, 0.0]
1/1 [==============================] - 0s 23ms/step
epoch: 201/10000, d_loss: 1.0675932317972183, d_acc: 0.6875, g_loss: [2.6318743228912354, 0.40625]
1/1 [==============================] - 0s 22ms/step
epoch: 301/10000, d_loss: 0.797086775302887, d_acc: 0.609375, g_loss: [1.6671888828277588, 0.40625]
1/1 [==============================] - 0s 15ms/step
epoch: 401/10000, d_loss: 0.7583125233650208, d_acc: 0.5, g_loss: [1.0700359344482422, 0.34375]
1/1 [==============================] - 0s 15ms/step
epoch: 501/10000, d_loss: 0.6772087812423706, d_acc: 0.53125, g_loss: [0.7200400233268738, 0.375]
1/1 [==============================] - 0s 16ms/step
epoch: 601/10000, d_loss: 0.6255382001399994, d_acc: 0.65625, g_loss: [0.8010768890380859, 0.21875]
1/1